# Importing the necessary library to process the Audio files and extract the features

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import librosa as lr
import re
from IPython.display import Audio
from scipy.fftpack import fft, rfft
from scipy import stats
from librosa.core import piptrack
from librosa.feature import mfcc
from scipy.stats.mstats import gmean

# Appending the directories for the audio files in a list, to open them later.

In [2]:
audio_files = glob('..//audios_to_test//recorded_voices//*.wav')

In [3]:
len(audio_files)

22

In [4]:
audio_files[0]

'..//audios_to_test//recorded_voices\\f1.wav'

## Opening the audio files using librosa, the autputs are:
- audio = audio time series
- sampling_rate = sampling rate of audio

In [5]:
# TO open an audio file at a time and listen to it in the notebook
audio, sampling_rate = lr.load(audio_files[18])
Audio(audio, rate=sampling_rate)

## To plot each audio file in the time domain

In [6]:
for file in range(0, len(audio_files), 1):
    audio, sampling_rate = lr.load(audio_files[file])
    time = np.arange(0, len(audio)) / sampling_rate
    #print(audio_files[file])
    #plt.plot(time, audio)
    #plt.xlabel('Time (secs)')
    #plt.show()

# Creating a function to extract the features needed out of an Audio file and then append the features into a dictionary

In [7]:
def feature_extractor(audio: np.ndarray, sampling_rate: int) -> dict:
    frequency_spectrum = np.abs(np.fft.rfft(audio))
    frequency = np.fft.rfftfreq(len(audio), d=1 / sampling_rate)
    frequency_spectrum = np.abs(frequency_spectrum)
    amplitude = frequency_spectrum / frequency_spectrum.sum()
    mean_frequency = (frequency * amplitude).sum()
    amplitude_cumulative_sum = np.cumsum(amplitude)
    median_frequency = frequency[len(amplitude_cumulative_sum[amplitude_cumulative_sum <= 0.5]) + 1]
    mode_frequency = frequency[amplitude.argmax()]
    quartile_25 = frequency[len(amplitude_cumulative_sum[amplitude_cumulative_sum <= 0.25]) + 1]
    deviation_from_mean = amplitude - amplitude.mean()
    amplitude_std = amplitude.std()
    kurtosis = ((deviation_from_mean ** 4).sum() / (len(frequency_spectrum) - 1)) / amplitude_std ** 4
    centroid_frequency = lr.feature.spectral_centroid(y=audio, sr=sampling_rate)
    spectral_flatness = lr.feature.spectral_flatness(y=audio)
    pitches, magnitudes = piptrack(y=audio, sr=sampling_rate, fmax = 280)
    mfccs = mfcc(y=audio, sr=sampling_rate)
    root_mean_square = lr.feature.rms(audio)

    dictionary_of_features = {
        'Mean_freq': mean_frequency/1000,
        'Mode_freq': mode_frequency/1000,
        'First_quartile': quartile_25/1000,
        'Kurtosis': kurtosis,
        'Centroid_freq': np.mean(centroid_frequency)/1000,
        'Spectral_flatness_measure': np.mean(spectral_flatness),
        'Mean_fundamental_freq': (pitches[np.nonzero(pitches)].mean())/1000,
        'Mean_MFCCs' : mfccs.mean(),                            #Mel-frequency cepstral coefficients (MFCCs)
        'Std_MFCCs': mfccs.std(),
        'Mean_Magnitude' : magnitudes.mean(), 
        'Max_MFCCs' : mfccs.max(),
        'Min_RMS': root_mean_square.min(),
        'Std_RMS': root_mean_square.std(),
    }

    return dictionary_of_features

# To open all the audio files, plot the frequency domain of ech file and apply the feature_extractor function to pull the features from the audio files and then append them into a list to use in creating a dataframe later.

### I added a condition to the loop to use the file name and append a new key: values pair in the dictionary, if the file starts with 'f' it appends 1 to a Gender key in the dictionary, if the file starts with anything else 'm', it appends 0 to the Gender key in the dictionary. 

In [8]:
list_of_dict = []
for file in range(0, len(audio_files), 1):
    audio, sampling_rate = lr.load(audio_files[file])
    #frequencies = rfft(audio)
    #plt.plot(abs(frequencies))
    #plt.show()
    #print(audio_files[file])
    dictionary_of_features = feature_extractor(audio, sampling_rate)
    if audio_files[file].startswith('..//audios_to_test//recorded_voices\\f'):
        dictionary_of_features['Gender'] = 1
    elif  audio_files[file].startswith('..//audios_to_test//recorded_voices\\m'):
        dictionary_of_features['Gender'] = 0
    list_of_dict.append(dictionary_of_features)

## Printing the list of dictionaries to make sure that all the dictionaries are in it.

In [9]:
#list_of_dict

## Creating a dataframe for the features extracted from the audio files

In [10]:
recorded_voices = pd.DataFrame(list_of_dict)

In [11]:
recorded_voices.head()

,Centroid_freq,First_quartile,Gender,Kurtosis,Max_MFCCs,Mean_MFCCs,Mean_Magnitude,Mean_freq,Mean_fundamental_freq,Min_RMS,Mode_freq,Spectral_flatness_measure,Std_MFCCs,Std_RMS
0,1.520965,0.420445,1,63.349399,189.586868,-25.048574,0.013635,1.620699,0.208515,0.00000,0.370466,0.004254,84.907509,0.026995
1,2.378998,0.667675,1,25.843574,151.076324,-21.781908,0.000263,2.227307,0.244304,0.00000,0.385132,0.003572,65.220032,0.042704
2,1.062615,0.410665,1,61.259496,235.741531,-25.663414,0.002186,0.941916,0.208736,0.00037,0.407236,0.000103,122.994820,0.009543
3,1.567703,0.468019,1,51.761446,197.113327,-23.402885,0.013373,1.549518,0.218778,0.00000,0.477835,0.007229,84.701729,0.038431
4,1.791400,0.499756,1,127.931409,190.621994,-25.624180,0.010491,1.789895,0.215905,0.00000,0.205834,0.004707,87.612541,0.027782


In [13]:
recorded_voices.to_csv('..//datasets//recorded_voices.csv', index=False)